In [1]:
import numpy as np
import cv2
import mediapipe as mp
import time
import tkinter as tk
from PIL import Image, ImageTk

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.1, model_complexity=2)

In [3]:
def calculate_angles(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180:
        angle=360-angle
    return round(angle,2)

def image_result(image):
    image=cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    image.flags.writeable=False
    results = pose.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    try:
        landmarks=results.pose_landmarks.landmark
        left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        angle=calculate_angles(left_shoulder,left_elbow,left_wrist)
        cv2.putText(image, f"{angle}", (int(left_elbow[0] * image.shape[1])-20, int(left_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (300, 300, 300), 1, cv2.LINE_AA)
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        angle = calculate_angles(right_shoulder, right_elbow, right_wrist)
        cv2.putText(image, f"{angle}", (int(right_elbow[0] * image.shape[1])-30, int(right_elbow[1] * image.shape[0])),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (300, 300, 300), 1, cv2.LINE_AA)
    except:
        pass
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(100, 100, 100), thickness=2, circle_radius=1),
        mp_drawing.DrawingSpec(color=(100, 0, 0), thickness=1)
    )

    return cv2.imwrite("image.png",image)


In [4]:
def calculate_angles(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180:
        angle=360-angle
    return round(angle,2)

In [11]:
cap = cv2.VideoCapture(0)
start_time = time.time()

root = tk.Tk()
root.title("Pose Detection")
screen_width=root.winfo_screenwidth()
screen_height=root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}")

text_label = tk.Label(root, text="Yoga Pose Correction",font=("Times New Roman",50))
text_label.place(x=340, y=10)


correct_images="pranamasana.png"
correct_images=image_result(correct_images)
correct_images="image.png"
correct_image=Image.open(correct_images)
correct_image_tk=ImageTk.PhotoImage(correct_image)

image_label=tk.Label(root,image=correct_image_tk)
image_label.place(x=200,y=140)

canvas = tk.Canvas(root, width=640, height=480)
canvas.place(x=600,y=100)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
        landmarks=results.pose_landmarks.landmark

        left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        left_arm_angle=calculate_angles(left_shoulder,left_elbow,left_wrist)
        cv2.putText(image, f"{left_arm_angle}", (int(left_elbow[0] * image.shape[1]), int(left_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (300, 300, 300), 3, cv2.LINE_AA)
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        right_arm_angle = calculate_angles(right_shoulder, right_elbow, right_wrist)

        cv2.putText(image, f"{right_arm_angle}", (int(right_elbow[0] * image.shape[1]-50), int(right_elbow[1] * image.shape[0])),
            cv2.FONT_HERSHEY_SIMPLEX, 1.1, (300, 300, 300), 3, cv2.LINE_AA)
       
    except:
        pass

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(232, 231, 21), thickness=2, circle_radius=3),
        mp_drawing.DrawingSpec(color=(0, 230, 0), thickness=1)
    )

    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(image)
    img_tk = ImageTk.PhotoImage(image=img)
    
    canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
    root.update()

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
    # current_time = time.time()
    # if current_time - start_time >= 5:
    #     screenshot = image.copy()  # Capture the frame directly
    #     cv2.imwrite("screenshot.png", screenshot)
    #     break

cap.release()
cv2.destroyAllWindows()
root.mainloop()


Exception ignored in: <function PhotoImage.__del__ at 0x00000181248895E0>
Traceback (most recent call last):
  File "c:\Users\25bak\anaconda3\lib\site-packages\PIL\ImageTk.py", line 146, in __del__
    name = self.__photo.name
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'


RuntimeError: Too early to create image: no default root window

In [17]:
import csv
import time
# List of keypoint names in the order provided by the documentation
keypoint_names = [
    "nose",
    "left_eye_inner", "left_eye", "left_eye_outer",
    "right_eye_inner", "right_eye", "right_eye_outer",
    "left_ear", "right_ear",
    "mouth_left", "mouth_right",
    "left_shoulder", "right_shoulder",
    "left_elbow", "right_elbow",
    "left_wrist", "right_wrist",
    "left_pinky", "right_pinky",
    "left_index", "right_index",
    "left_thumb", "right_thumb",
    "left_hip", "right_hip",
    "left_knee", "right_knee",
    "left_ankle", "right_ankle",
    "left_heel", "right_heel",
    "left_foot_index", "right_foot_index"
]

# Create a CSV file and write the header row
csv_filename = "keypoint_landmarks.csv"
with open(csv_filename, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    header = ["KeyPointName"] + ["X"] + ["Y"]
    csv_writer.writerow(header)

cap=cv2.VideoCapture(0)
start_time=time.time()
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
        landmarks = results.pose_landmarks.landmark
        landmarks_data = []
        
        for idx, landmark in enumerate(landmarks):
            landmarks_data.append([frame_count] + [landmark.x, landmark.y])
       
    except:
        pass

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(232, 231, 21), thickness=2, circle_radius=3),
        mp_drawing.DrawingSpec(color=(0, 230, 0), thickness=1)
    )

    # Display the processed frame
    cv2.imshow("Real-Time Video", image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    end_time=time.time()
    if end_time-start_time>5:
        last_frame_landmarks=landmarks_data
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Append the landmarks data to the CSV file
with open(csv_filename, mode="a", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(last_frame_landmarks)


In [9]:
# tk.Label?

In [10]:
# def calculate_angles(a,b,c):
#     a=np.array(a)
#     b=np.array(b)
#     c=np.array(c)
#     radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
#     angle=np.abs(radians*180.0/np.pi)
#     if angle>180:
#         angle=360-angle
#     return round(angle,2)

In [11]:
# import numpy as np
# import cv2
# import mediapipe as mp
# from PIL import Image

# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(min_detection_confidence=0.1, model_complexity=2)

# def calculate_angles(a, b, c):
#     a = np.array(a)
#     b = np.array(b)
#     c = np.array(c)
#     radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
#     angle = np.abs(radians * 180.0 / np.pi)
#     if angle > 180:
#         angle = 360 - angle
#     return round(angle, 2)

# def image_result(image, correct_image):
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image to BGR format
#     image.flags.writeable = False
#     results = pose.process(image)
#     image.flags.writeable = True
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     try:
#         landmarks = results.pose_landmarks.landmark
#         left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
#                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
#         left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
#                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
#         left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
#                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
#         angle = calculate_angles(left_shoulder, left_elbow, left_wrist)
#         cv2.putText(image, f"{angle}", (int(left_elbow[0] * image.shape[1]) - 20,
#                                          int(left_elbow[1] * image.shape[0])),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
        
#         right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                           landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
#         right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
#                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
#         right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
#                        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
#         angle = calculate_angles(right_shoulder, right_elbow, right_wrist)
#         cv2.putText(image, f"{angle}", (int(right_elbow[0] * image.shape[1]) - 30,
#                                          int(right_elbow[1] * image.shape[0])),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
#     except:
#         pass

#     mp_drawing.draw_landmarks(
#         image,
#         results.pose_landmarks,
#         mp_pose.POSE_CONNECTIONS,
#         mp_drawing.DrawingSpec(color=(200, 0, 0), thickness=2, circle_radius=1),
#         mp_drawing.DrawingSpec(color=(200, 0, 0), thickness=1)
#     )
    
#     # Display both the corrected image and the pose landmarks
#     correct_image = correct_image.resize((image.shape[1], image.shape[0]))
#     correct_image = cv2.cvtColor(np.array(correct_image), cv2.COLOR_RGB2BGR)
#     combined_image = cv2.addWeighted(image, 0.5, correct_image, 0.5, 0)
    
#     cv2.imshow("Image with Pose Landmarks and Corrected Image", combined_image)
#     cv2.waitKey(0) & 0xFF == ord("q")
#     cv2.destroyAllWindows()

# # Load the correct image
# correct_image_path = "pranamasana.png"
# correct_image = Image.open(correct_image_path)

# # Open the camera
# cap = cv2.VideoCapture(0)
# start_time = time.time()

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         continue

#     image_result(frame, correct_image)

#     current_time = time.time()
#     if current_time - start_time >= 5:
#         screenshot = frame.copy()  # Capture the frame directly
#         cv2.imwrite("screenshot.png", screenshot)
#         break

# cap.release()
# cv2.destroyAllWindows()
